# Combining & organizing data

## Libraries and settings

In [24]:
# Libraries
import os
import pandas as pd
import numpy as np
import fnmatch

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

/workspaces/data_analytics/Week_03


## Importing the prepared rental apartments data

In [25]:
# Read data to pandas data frame
df_orig = pd.read_csv('apartments_data_prepared.csv', 
                      sep=',', 
                      encoding='utf-8')

# Copy of data with selected colums
columns = [ 'web-scraper-order', 
            'address_raw',
            'rooms', 
            'area', 
            'luxurious',
            'price',
            'price_per_m2']
df = df_orig[columns]

# Get number of rows and columns
print(df.shape)

# Show first records
df.head(5)

(865, 7)


,web-scraper-order,address_raw,rooms,area,luxurious,price,price_per_m2
0,1693998201-1,"Neuhusstrasse 6, 8630 Rüti ZH, ZH",3.0,49.0,0,1441.0,29.41
1,1693998201-2,"Zürcherstrasse 1, 8173 Neerach, ZH",3.5,65.0,0,1850.0,28.46
2,1693998201-4,"Cramerstrasse 8-12, 8004 Zürich, ZH",2.0,54.0,0,4853.0,89.87
3,1693998201-5,"Rotachstrasse 33, 8003 Zürich, ZH",2.0,49.0,0,4335.0,88.47
4,1693998201-16,"Wolframplatz 1, 8045 Zürich, ZH",2.0,32.0,0,3515.0,109.84


## Importing municipality data


In [26]:
# Read data to pandas data frame
df_municipality_taxable = pd.read_csv('municipality_data.csv', 
                      sep=',', 
                      encoding='utf-8')

# Copy of data with selected colums
columns = [ 'bfs_number', 
            'bfs_name',
            'pop', 
            'pop_dens', 
            'frg_pct',
            'emp',
            'mean_taxable_income']
df = df_municipality_taxable[columns]

# Get number of rows and columns
print(df.shape)

# Show first records
df.head(5)

(2172, 7)


,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp,mean_taxable_income
0,1,Aeugst am Albis,1981,250.442478,14.184755,442.0,CHF 105'013.57
1,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,CHF 71'059.81
2,3,Bonstetten,5572,749.932705,16.564968,1014.0,CHF 88'927.70
3,4,Hausen am Albis,3751,275.808824,16.022394,1021.0,CHF 86'300.46
4,5,Hedingen,3778,578.560490,16.410799,1478.0,CHF 90'811.21


## Combining data from different sources

### Reading rental apartment data with geocoded addresses

In [27]:
# Meaning of variables
# lat: geographical latitude
# lon: geographical longitude
# bfs_number: official municipality id
# bfs_name: official municipality name

# Geocoded data (i.e. data with latitude and longitude)
df_geo = pd.read_csv('apartments_data_geocoded.csv', 
                     sep=',', 
                     encoding='utf-8')

# Rename columns
df_geo = df_geo.rename(columns={'web-scrape':'web-scraper-order',
                                'address_ra':'address_raw',
                                'BFS_NUMMER':'bfs_number', 
                                'NAME':'bfs_name'})

# Show data
df_geo[['web-scraper-order',
        'address_raw',
        'lat', 
        'lon', 
        'bfs_number', 
        'bfs_name']].head()

,web-scraper-order,address_raw,lat,lon,bfs_number,bfs_name
0,1693998201-1,"Neuhusstrasse 6, 8630 Rüti ZH, ZH",47.252171,8.845797,118,Rüti (ZH)
1,1693998201-2,"Zürcherstrasse 1, 8173 Neerach, ZH",47.513332,8.474851,88,Neerach
2,1693998201-5,"Rotachstrasse 33, 8003 Zürich, ZH",47.370792,8.514748,261,Zürich
3,1693998201-16,"Wolframplatz 1, 8045 Zürich, ZH",47.362282,8.522193,261,Zürich
4,1693998205-25,"Badenerstrasse 67, 8953 Dietikon, ZH",47.407925,8.392561,243,Dietikon


### Join geo-information to rental apartment data using .merge()

In [28]:
df2 = df.merge(df_geo[['web-scraper-order', 
                       'lat', 
                       'lon', 
                       'bfs_number', 
                       'bfs_name']], 
               on="bfs_number")
df2.head()

,bfs_number,bfs_name_x,pop,pop_dens,frg_pct,emp,mean_taxable_income,web-scraper-order,lat,lon,bfs_name_y
0,1,Aeugst am Albis,1981,250.442478,14.184755,442.0,CHF 105'013.57,1693998252-290,47.268188,8.485293,Aeugst am Albis
1,1,Aeugst am Albis,1981,250.442478,14.184755,442.0,CHF 105'013.57,1693998265-375,47.267799,8.486970,Aeugst am Albis
2,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,CHF 71'059.81,1693998228-154,47.275436,8.445224,Affoltern am Albis
3,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,CHF 71'059.81,1693998236-211,47.283363,8.453011,Affoltern am Albis
4,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,CHF 71'059.81,1693998272-386,47.278049,8.454066,Affoltern am Albis


### Reading municipality-level data

In [29]:
# Meaning of variables:
# bfs_number: official municipality id
# bfs_name: official municipality name
# pop: number of residents (=population)
# pop_dens: population density (pop per km2)
# frg_pct: percentage foreigners
# emp: numer of employees

df_municip = pd.read_excel('municipality_data.xlsx', 
                           sheet_name='data_for_import')
df_municip.head(5)

,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp
0,1,Aeugst am Albis,1981,250.442478,14.184755,442.0
1,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0
2,3,Bonstetten,5572,749.932705,16.564968,1014.0
3,4,Hausen am Albis,3751,275.808824,16.022394,1021.0
4,5,Hedingen,3778,578.560490,16.410799,1478.0


### Join municipality data to rental apartment data using .merge()

In [30]:
# Merge needs a key which must be identical in both data sets (here the key is 'bfs_number')
df3 = df2.merge(df_municip[['bfs_number', 
                            'pop', 
                            'pop_dens', 
                            'frg_pct', 
                            'emp']], 
                on="bfs_number")
df3.head(5)

,bfs_number,bfs_name_x,pop_x,pop_dens_x,frg_pct_x,emp_x,mean_taxable_income,web-scraper-order,lat,lon,bfs_name_y,pop_y,pop_dens_y,frg_pct_y,emp_y
0,1,Aeugst am Albis,1981,250.442478,14.184755,442.0,CHF 105'013.57,1693998252-290,47.268188,8.485293,Aeugst am Albis,1981,250.442478,14.184755,442.0
1,1,Aeugst am Albis,1981,250.442478,14.184755,442.0,CHF 105'013.57,1693998265-375,47.267799,8.486970,Aeugst am Albis,1981,250.442478,14.184755,442.0
2,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,CHF 71'059.81,1693998228-154,47.275436,8.445224,Affoltern am Albis,12303,1161.756374,28.700317,6920.0
3,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,CHF 71'059.81,1693998236-211,47.283363,8.453011,Affoltern am Albis,12303,1161.756374,28.700317,6920.0
4,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,CHF 71'059.81,1693998272-386,47.278049,8.454066,Affoltern am Albis,12303,1161.756374,28.700317,6920.0


### Join additional variable to the apartment data using .merge()

In [31]:
df3 = df2.merge(df_municipality_taxable[['bfs_number',
                            'mean_taxable_income']], 
                on="bfs_number")
df3.head(5)

,bfs_number,bfs_name_x,pop,pop_dens,frg_pct,emp,mean_taxable_income_x,web-scraper-order,lat,lon,bfs_name_y,mean_taxable_income_y
0,1,Aeugst am Albis,1981,250.442478,14.184755,442.0,CHF 105'013.57,1693998252-290,47.268188,8.485293,Aeugst am Albis,CHF 105'013.57
1,1,Aeugst am Albis,1981,250.442478,14.184755,442.0,CHF 105'013.57,1693998265-375,47.267799,8.486970,Aeugst am Albis,CHF 105'013.57
2,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,CHF 71'059.81,1693998228-154,47.275436,8.445224,Affoltern am Albis,CHF 71'059.81
3,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,CHF 71'059.81,1693998236-211,47.283363,8.453011,Affoltern am Albis,CHF 71'059.81
4,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,CHF 71'059.81,1693998272-386,47.278049,8.454066,Affoltern am Albis,CHF 71'059.81


### Export data to file

In [32]:
# Remove missing values which may have ocured from the merging process
df3 = df3.dropna()

# Check for missing values per column
print(df3.isna().sum())

# Count number of rows and columns
print(df3.shape)

# Export apartment data to file
df3.to_csv('apartments_data_enriched.csv',
            sep=',',
            encoding='utf-8')

bfs_number               0
bfs_name_x               0
pop                      0
pop_dens                 0
frg_pct                  0
emp                      0
mean_taxable_income_x    0
web-scraper-order        0
lat                      0
lon                      0
bfs_name_y               0
mean_taxable_income_y    0
dtype: int64
(786, 12)


### Sorting data

In [35]:
# Sorting data by 'price' and 'area' with highest price above (ascending=False)
df3.sort_values(by=['price', 'area'], 
                ascending=False).head()

KeyError: 'price'

### Aggregation of data

In [34]:
# Aggregation using .groupby()
df3[['rooms', 'price']].groupby(['rooms']).mean()

KeyError: "None of [Index(['rooms', 'price'], dtype='object')] are in the [columns]"

## Reshaping data

### Reshaping data using .stack() and .unstack()

In [ ]:
df_sub = df3[['bfs_name', 'rooms', 'price', 'area']][:5]
print('Original shape')
print(df_sub, '\n')

df_sub_stacked = df_sub.stack()
print('Stacked')
print(df_sub_stacked, '\n')

# Using unstack
print('Unstacked (= back to original shape)')
print(df_sub_stacked.unstack())

Original shape
    bfs_name  rooms   price   area
0  Rüti (ZH)    3.0  1441.0   49.0
1  Rüti (ZH)    3.0  2600.0  111.0
2  Rüti (ZH)    3.0  1490.0   58.0
3  Rüti (ZH)    4.0  3240.0  118.0
4  Rüti (ZH)    3.0  1450.0   66.0 

Stacked
0  bfs_name    Rüti (ZH)
   rooms             3.0
   price          1441.0
   area             49.0
1  bfs_name    Rüti (ZH)
   rooms             3.0
   price          2600.0
   area            111.0
2  bfs_name    Rüti (ZH)
   rooms             3.0
   price          1490.0
   area             58.0
3  bfs_name    Rüti (ZH)
   rooms             4.0
   price          3240.0
   area            118.0
4  bfs_name    Rüti (ZH)
   rooms             3.0
   price          1450.0
   area             66.0
dtype: object 

Unstacked (= back to original shape)
    bfs_name rooms   price   area
0  Rüti (ZH)   3.0  1441.0   49.0
1  Rüti (ZH)   3.0  2600.0  111.0
2  Rüti (ZH)   3.0  1490.0   58.0
3  Rüti (ZH)   4.0  3240.0  118.0
4  Rüti (ZH)   3.0  1450.0   66.0


### Reshaping data using .melt()

In [ ]:
df_sub = df3[['rooms', 'price', 'area']][:5]
print('Original shape')
print(df_sub, '\n')

print('Reshaped using .melt()')
print(pd.melt(df, id_vars=['rooms'], value_vars=['price', 'area']))

Original shape
   rooms   price   area
0    3.0  1441.0   49.0
1    3.0  2600.0  111.0
2    3.0  1490.0   58.0
3    4.0  3240.0  118.0
4    3.0  1450.0   66.0 

Reshaped using .melt()
      rooms variable   value
0       3.0    price  1441.0
1       3.5    price  1850.0
2       2.0    price  4853.0
3       2.0    price  4335.0
4       2.0    price  3515.0
...     ...      ...     ...
1725    3.5     area    82.0
1726    4.0     area    73.0
1727    4.5     area   110.0
1728    3.0     area    68.0
1729    4.5     area   114.0

[1730 rows x 3 columns]


### Pivoting data using .pivot_table()

In [ ]:
pivot_table = pd.pivot_table(df3[['rooms', 'price', 'area', 'price_per_m2']],
               index=['rooms'],
               values=['price', 'area', 'price_per_m2'],
               aggfunc=(np.mean))

# price_per_m2 becomes lower the higher area
pivot_table


,area,price,price_per_m2
rooms,,,
1.0,32.057692,1306.346154,50.168077
1.5,35.972973,2092.054054,64.345676
2.0,54.475000,2138.775000,42.201750
2.5,67.340278,2414.993056,37.184861
3.0,68.574468,1942.659574,28.307447
3.5,89.500000,2664.813084,29.711355
4.0,96.900000,2798.200000,29.482000
4.5,114.587209,3073.290698,26.489186
5.0,103.500000,2773.750000,26.587500


In [ ]:



mean_price_per_m2 = pivot_table['price_per_m2']

comparison_df = pd.DataFrame({'Mean Price': mean_price, 'Mean Price per m2': mean_price_per_m2})
comparison_df

# price_per_m2 becomes lower the higher area

,Mean Price,Mean Price per m2
rooms,,
1.0,1306.346154,50.168077
1.5,2092.054054,64.345676
2.0,2138.775000,42.201750
2.5,2414.993056,37.184861
3.0,1942.659574,28.307447
3.5,2664.813084,29.711355
4.0,2798.200000,29.482000
4.5,3073.290698,26.489186
5.0,2773.750000,26.587500


In [ ]:
pivot_table_counts = pd.pivot_table(df3,
                                    index=['rooms'],
                                    values=['price'],
                                    aggfunc='count')

pivot_table_counts

,price
rooms,
1.0,52
1.5,37
2.0,40
2.5,144
3.0,47
3.5,214
4.0,30
4.5,172
5.0,4


In [ ]:
pivot_table = pd.pivot_table(df3[['rooms', 'price', 'area', 'price_per_m2', 'bfs_name']],
               index=['bfs_name'],
               values=['rooms', 'area', 'price', 'price_per_m2'],
               aggfunc=np.mean)

sorted_pivot_table = pivot_table.sort_values(by=['price', 'area'], ascending=[False, False])
sorted_pivot_table


,area,price,price_per_m2,rooms
bfs_name,,,,
Rüschlikon,144.000000,6660.000000,46.250000,4.5
Küsnacht (ZH),136.333333,4891.666667,35.476667,4.5
Nürensdorf,506.500000,4500.000000,17.605000,6.0
Maur,138.500000,4250.000000,30.445000,4.5
Obfelden,127.500000,4095.000000,31.910000,5.5
...,...,...,...,...
Weiningen (ZH),73.333333,1606.666667,20.716667,3.0
Dättlikon,90.000000,1480.000000,16.440000,4.0
Weiach,46.000000,1420.000000,30.870000,1.5


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 5.15.0-1041-azure
Datetime: 2023-10-10 17:31:14
Python Version: 3.10.13
-----------------------------------
